In [ ]:
import sys; sys.path.append('..')
from common.postgresql import PostgresConnector
db = PostgresConnector()
import os
import requests
import zipfile
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# Configurações
BASE_URL = "https://dados.cvm.gov.br/dados/"
DEST_DIR = r"E:/Download/cvm"

# Inicializa banco de dados
connector = db

def setup_control_table():
    """Garante que a tabela de controle exista antes de iniciar."""
    # Cria um DF vazio com a estrutura necessária para o controle
    df_schema = pd.DataFrame(columns=['file_name', 'relative_path', 'last_download'])
    # create_table está blindada: se a tabela existir, ela não faz nada
    connector.create_table(df_schema, 'cvm.ingest_control')

def check_should_skip(file_name, relative_path):
    """
    Verifica se deve pular o arquivo.
    Prioridade 1: Banco de dados (últimos 7 dias)
    Prioridade 2: Existência do arquivo físico
    """
    # 1. Verifica no Banco de Dados
    # O query busca se o arquivo foi baixado nos últimos 7 dias
    query = f"""
        SELECT 1 FROM cvm.ingest_control 
        WHERE file_name = '{file_name}' 
        AND relative_path = '{relative_path}'
        AND last_download > CURRENT_DATE - INTERVAL '7 days'
    """
    res = connector.read_sql(query)
    if not res.empty:
        return True

    # 2. Verifica na Pasta Destino
    # Se o CSV correspondente já existe, pula também
    local_csv_path = os.path.join(DEST_DIR, relative_path).replace('.zip', '.csv')
    if os.path.exists(local_csv_path):
        return True

    return False

def register_download(file_name, relative_path):
    """Registra o sucesso do download na tabela de controle."""
    df_log = pd.DataFrame([{
        'file_name': file_name,
        'relative_path': relative_path,
        'last_download': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }])
    # Upsert garante que se o arquivo for baixado novamente após 7 dias, a data seja atualizada
    connector.upsert_dataframe(df_log, 'cvm.ingest_control', logical_pks=['file_name', 'relative_path'])

# Conjunto para rastrear o que já foi mapeado e evitar loops
visited_urls = set()

def is_valid_url(url):
    parsed = urlparse(url)
    return parsed.netloc == "dados.cvm.gov.br" and parsed.path.startswith("/dados/")

def get_content(url):
    try:
        if url in visited_urls or not is_valid_url(url):
            return [], []
        
        visited_urls.add(url)
        response = requests.get(url, timeout=15)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        links = [node.get('href') for node in soup.find_all('a') if node.get('href')]
        pastas, arquivos = [], []
        
        for link in links:
            if link in ['../', './', '/'] or link.startswith('?') or link.startswith('http'):
                if not link.startswith('http'): pass 
                else:
                    if not is_valid_url(link): continue

            full_url = urljoin(url, link)
            if link.endswith('/'):
                if full_url not in visited_urls:
                    pastas.append(full_url)
            elif link.lower().endswith(('.zip', '.csv')):
                arquivos.append(full_url)
                
        return pastas, arquivos
    except Exception as e:
        print(f"Erro ao acessar {url}: {e}")
        return [], []

def download_and_unzip(url):
    relative_path = urlparse(url).path.replace("/dados/", "").lstrip("/")
    file_name = url.split('/')[-1]
    
    # VERIFICAÇÃO DE PULO
    if check_should_skip(file_name, relative_path):
        # Omiti o print para não poluir o terminal, mas o crawler continua
        return

    sub_folders = os.path.split(relative_path)[0]
    local_dir = os.path.join(DEST_DIR, sub_folders)
    os.makedirs(local_dir, exist_ok=True)
    file_path = os.path.join(local_dir, file_name)

    print(f"Processando: {relative_path}")
    try:
        r = requests.get(url, stream=True)
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024*1024):
                f.write(chunk)
        
        if file_name.lower().endswith('.zip'):
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(local_dir)
            os.remove(file_path)
        
        # REGISTRO NO BANCO APÓS SUCESSO
        register_download(file_name, relative_path)
        
    except Exception as e:
        print(f"Falha em {file_name}: {e}")

def run_crawler(start_url):
    # Passo 0: Garantir tabela de controle
    setup_control_table()
    
    stack = [start_url]
    while stack:
        current_url = stack.pop()
        sub_pastas, arquivos = get_content(current_url)
        
        for arq in arquivos:
            download_and_unzip(arq)
            
        for pasta in sub_pastas:
            stack.append(pasta)

print(f"Iniciando Crawling em {BASE_URL}...")
run_crawler(BASE_URL)
print("\nFinalizado!")